In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import pandas as pd

PATH = ("/content/drive/MyDrive/Spotify_Project/4_Feature_Engineering/spotify_with_features.pkl")

with open(PATH, "rb") as f:
    DF = pickle.load(f)

print("Loaded:", DF.shape)
display(DF.head())

Loaded: (32833, 57)


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,has_with,has_dont,has_remastered,has_version,has_like,has_your,has_original,has_remaster,has_down,has_this
0,6f807x0ima9a1j3VPbc7VN,i dont care with justin bieber loud luxury remix,ed sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,i dont care with justin bieber loud luxury remix,2019-06-14,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,1,0,0,0,0,0,0,0,0
1,0r7CVbZTWZgbTCYdfa2P31,memories dillon francis remix,maroon 5,67,63rPSO264uRjW1X5E6cWv6,memories dillon francis remix,2019-12-13,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
2,1z1Hg7Vb0AhHDiEmnDE79l,all the time don diablo remix,zara larsson,70,1HoSmj2eLcsrR0vE9gThr4,all the time don diablo remix,2019-07-05,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
3,75FpbthrwQmzHlBJLuGdC7,call you mine keanu silva remix,the chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,call you mine the remixes,2019-07-19,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0
4,1e8PAfcKUYoKkxPhrHqw4x,someone you loved future humans remix,lewis capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,someone you loved future humans remix,2019-03-05,pop remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_final = DF.copy()
print("Working copy created:", df_final.shape)

Working copy created: (32833, 57)


In [5]:
from sklearn.model_selection import train_test_split

# Work on the modeling copy
X = df_final.drop(columns=["track_popularity"])
y = df_final["track_popularity"]

# Step 1: Train (70%) vs Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, train_size=0.70, random_state=222, shuffle=True
)

# Step 2: split Temp into Dev (15%) and Test (15%)
X_dev, X_test, y_dev, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=222, shuffle=True
)

print(f"Train: {len(X_train)} ({len(X_train)/len(df_final):.1%})")
print(f"Dev:   {len(X_dev)} ({len(X_dev)/len(df_final):.1%})")
print(f"Test:  {len(X_test)} ({len(X_test)/len(df_final):.1%})")

Train: 22983 (70.0%)
Dev:   4925 (15.0%)
Test:  4925 (15.0%)


In [6]:
cols_to_check = [
    "track_name", "track_artist", "track_album_name",
    "playlist_name", "playlist_subgenre"
]

total_count = len(X_train)

for col in [c for c in cols_to_check if c in X_train.columns]:
    unique_count = X_train[col].nunique(dropna=False)
    print(f"{col:25} → {unique_count:6} unique values ({unique_count/total_count:.1%} of total rows)")

track_name                →  17350 unique values (75.5% of total rows)
track_artist              →   8628 unique values (37.5% of total rows)
track_album_name          →  15011 unique values (65.3% of total rows)
playlist_name             →    423 unique values (1.8% of total rows)
playlist_subgenre         →     24 unique values (0.1% of total rows)


In [7]:
!pip install category_encoders

import pandas as pd
from category_encoders import LeaveOneOutEncoder

te_cols = ["track_artist", "playlist_name"]
count_cols = ["track_name", "track_artist", "track_album_name", "playlist_name"]

for c in count_cols:
    vc = X_train[c].value_counts()
    X_train[f"{c}_count"] = X_train[c].map(vc).fillna(0)
    X_dev[f"{c}_count"]   = X_dev[c].map(vc).fillna(0)
    X_test[f"{c}_count"]  = X_test[c].map(vc).fillna(0)

enc = LeaveOneOutEncoder(cols=te_cols, return_df=True, sigma=0.05)
X_train_te = enc.fit_transform(X_train[te_cols], y_train)
X_dev_te   = enc.transform(X_dev[te_cols])
X_test_te  = enc.transform(X_test[te_cols])

for c in te_cols:
    X_train[f"{c}_TE"] = X_train_te[c]
    X_dev[f"{c}_TE"]   = X_dev_te[c]
    X_test[f"{c}_TE"]  = X_test_te[c]
    X_train.drop(columns=[c], inplace=True)
    X_dev.drop(columns=[c], inplace=True)
    X_test.drop(columns=[c], inplace=True)

print("Done. Shapes:", X_train.shape, X_dev.shape, X_test.shape)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.8 MB/s eta 0:00:00
Done. Shapes: (22983, 60) (4925, 60) (4925, 60)


In [8]:
drop_cols = [
    "track_id", "playlist_id", "track_album_id",
    "track_name", "track_artist",
    "track_album_name", "playlist_name",
    "track_album_release_date"
]

X_train_model = X_train.drop(columns=drop_cols, errors="ignore").copy()
X_dev_model   = X_dev.drop(columns=drop_cols, errors="ignore").copy()
X_test_model  = X_test.drop(columns=drop_cols, errors="ignore").copy()

In [9]:
# Identify categorical columns left after TE
cat_cols = X_train_model.select_dtypes(include=["object", "category"]).columns
print("Categorical columns left:", list(cat_cols))

# Encode separately and align columns
X_train_enc = pd.get_dummies(X_train_model, columns=cat_cols, drop_first=True)
X_dev_enc   = pd.get_dummies(X_dev_model,   columns=cat_cols, drop_first=True)
X_test_enc  = pd.get_dummies(X_test_model,  columns=cat_cols, drop_first=True)

# Align columns so all sets match the train structure
X_dev_enc  = X_dev_enc.reindex(columns=X_train_enc.columns, fill_value=0)
X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

print("Shapes after encoding:", X_train_enc.shape, X_dev_enc.shape, X_test_enc.shape)

Categorical columns left: ['playlist_genre', 'playlist_subgenre', 'key', 'mode', 'release_dow', 'release_dow_name', 'mode_label']
Shapes after encoding: (22983, 100) (4925, 100) (4925, 100)


In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Train quick models on TRAIN only
lasso = Lasso(alpha=5, max_iter=10000, random_state=42).fit(X_train_enc, y_train)
ridge = Ridge(alpha=5, random_state=42).fit(X_train_enc, y_train)
gb    = GradientBoostingRegressor(random_state=42).fit(X_train_enc, y_train)
rf    = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1).fit(X_train_enc, y_train)

# Feature selection summary
lasso_sel = (np.abs(lasso.coef_) > 0).astype(int)
ridge_sel = (np.abs(ridge.coef_) > 0.05 * np.abs(ridge.coef_).max()).astype(int)
gb_sel    = (gb.feature_importances_ > gb.feature_importances_.mean()).astype(int)
rf_sel    = (rf.feature_importances_ > rf.feature_importances_.mean()).astype(int)

selection_df = pd.DataFrame({
    "Feature": X_train_enc.columns,
    "Lasso": lasso_sel,
    "Ridge": ridge_sel,
    "GradientBoost": gb_sel,
    "RandomForest": rf_sel
})

selection_df["Sum"] = selection_df[["Lasso","Ridge","GradientBoost","RandomForest"]].sum(axis=1)
selection_df = selection_df.sort_values("Sum", ascending=False)

print("\nTop 20 features by importance consensus:")
print(selection_df.head(20))


Top 20 features by importance consensus:
                  Feature  Lasso  Ridge  GradientBoost  RandomForest  Sum
45        track_artist_TE      1      1              1             1    4
46       playlist_name_TE      1      1              1             1    4
18       album_song_count      0      1              1             1    3
19    song_playlist_count      0      1              1             1    3
9            release_year      1      0              1             1    3
3             speechiness      0      1              0             1    2
12   instrumentalness_log      0      1              0             1    2
4            acousticness      0      1              0             1    2
11            release_day      1      0              0             1    2
8             duration_ms      1      0              0             1    2
1                  energy      0      1              0             1    2
5                liveness      0      1              0             1  

In [11]:
selected_features = selection_df.loc[selection_df["Sum"] >= 2, "Feature"].tolist()

# Extra subgenre features to add
extra_feats = [
    "playlist_subgenre_permanent wave",
    "playlist_subgenre_new jack swing",
    "playlist_subgenre_progressive electro house",
    "playlist_subgenre_hip hop",
    "playlist_subgenre_post-teen pop"
]

# Final feature list (unique & ordered)
final_features = list(dict.fromkeys(selected_features + extra_feats))

# Create reduced matrices for modeling
X_train_sel = X_train_enc[final_features].copy()
X_dev_sel   = X_dev_enc[final_features].copy()
X_test_sel  = X_test_enc[final_features].copy()

print(f"{len(final_features)} total features ({len(selected_features)} + {len(extra_feats)} extras)")
print("Shapes:", X_train_sel.shape, X_dev_sel.shape, X_test_sel.shape)

print("\nFinal features preview:", final_features)

21 total features (16 + 5 extras)
Shapes: (22983, 21) (4925, 21) (4925, 21)

Final features preview: ['track_artist_TE', 'playlist_name_TE', 'album_song_count', 'song_playlist_count', 'release_year', 'speechiness', 'instrumentalness_log', 'acousticness', 'release_day', 'duration_ms', 'energy', 'liveness', 'artist_playlist_count', 'track_artist_count', 'valence', 'danceability', 'playlist_subgenre_permanent wave', 'playlist_subgenre_new jack swing', 'playlist_subgenre_progressive electro house', 'playlist_subgenre_hip hop', 'playlist_subgenre_post-teen pop']


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_sel)
X_dev_scaled   = scaler.transform(X_dev_sel)
X_test_scaled  = scaler.transform(X_test_sel)

print("Scaled shapes:", X_train_scaled.shape, X_dev_scaled.shape, X_test_scaled.shape)

Scaled shapes: (22983, 21) (4925, 21) (4925, 21)


In [13]:
# Check data types of your selected feature matrix
X_train_sel.dtypes.value_counts()

,count
float64,10
bool,5
int64,3
Int64,3


In [14]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

HAS_XGB = False
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    pass

# Ensure scaled matrices exist
try:
    X_train_scaled, X_dev_scaled, X_test_scaled
except NameError:
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_sel)
    X_dev_scaled   = scaler.transform(X_dev_sel)
    X_test_scaled  = scaler.transform(X_test_sel)

# Define models
models = {
    "Linear": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1),
    "GradientBoost": GradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "KNeighbors": KNeighborsRegressor(n_neighbors=5),
    "SVR": SVR(kernel="rbf"),
}
if HAS_XGB:
    models["XGBoost"] = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1
    )

def _safe_rmsle(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    y_true_clip = np.clip(y_true, 0, None)
    y_pred_clip = np.clip(y_pred, 0, None)
    return np.sqrt(np.mean((np.log1p(y_pred_clip) - np.log1p(y_true_clip))**2))

def _all_metrics(y_true, y_pred):
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    rmsle = _safe_rmsle(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    return mse, rmse, mae, rmsle, r2

rows = []
for name, model in models.items():
    scale_sensitive = name in ["Linear", "KNeighbors", "SVR"]

    Xtr = X_train_scaled if scale_sensitive else X_train_sel
    Xdv = X_dev_scaled   if scale_sensitive else X_dev_sel
    Xts = X_test_scaled  if scale_sensitive else X_test_sel

    model.fit(Xtr, y_train)

    pred_tr = model.predict(Xtr)
    pred_dv = model.predict(Xdv)
    pred_ts = model.predict(Xts)

    tr_mse, tr_rmse, tr_mae, tr_rmsle, tr_r2 = _all_metrics(y_train, pred_tr)
    dv_mse, dv_rmse, dv_mae, dv_rmsle, dv_r2 = _all_metrics(y_dev,   pred_dv)
    ts_mse, ts_rmse, ts_mae, ts_rmsle, ts_r2 = _all_metrics(y_test,  pred_ts)

    rows.append({
        "Model": name,
        "Train_MSE": tr_mse, "Train_RMSE": tr_rmse, "Train_MAE": tr_mae, "Train_RMSLE": tr_rmsle, "Train_R2": tr_r2,
        "Dev_MSE":   dv_mse, "Dev_RMSE":   dv_rmse, "Dev_MAE":   dv_mae, "Dev_RMSLE":   dv_rmsle, "Dev_R2":   dv_r2,
        "Test_MSE":  ts_mse, "Test_RMSE":  ts_rmse, "Test_MAE":  ts_mae, "Test_RMSLE":  ts_rmsle, "Test_R2":  ts_r2,
    })

results_all = pd.DataFrame(rows).sort_values("Dev_R2", ascending=False)
pd.set_option("display.precision", 4)
print(results_all.to_string(index=False))

        Model  Train_MSE  Train_RMSE  Train_MAE  Train_RMSLE  Train_R2  Dev_MSE  Dev_RMSE  Dev_MAE  Dev_RMSLE  Dev_R2  Test_MSE  Test_RMSE  Test_MAE  Test_RMSLE  Test_R2
 RandomForest    38.5203      6.2065     4.6388       0.6925    0.9380 277.6160   16.6618  12.6014     1.0372  0.5514  301.2883    17.3577   13.1677      1.0675   0.5305
GradientBoost   290.0070     17.0296    13.1082       1.0813    0.5334 303.2458   17.4140  13.7438     1.0618  0.5100  329.2273    18.1446   14.3897      1.0942   0.4870
      XGBoost   159.1688     12.6162     9.4726       0.9264    0.7439 303.6477   17.4255  13.5109     1.0374  0.5093  329.2069    18.1441   14.1592      1.0702   0.4870
       Linear   323.2321     17.9787    14.0443       1.1225    0.4800 317.4664   17.8176  14.0797     1.1156  0.4870  339.4738    18.4248   14.5807      1.1446   0.4710
          SVR   326.5376     18.0704    13.3068       1.1453    0.4746 322.0395   17.9455  13.5135     1.1386  0.4796  348.0802    18.6569   14.0803  


Although RandomForest initially achieved the highest R² on the training set (0.94), it showed signs of overfitting.
XGBoost demonstrated more balanced performance (Train R² = 0.74, Dev R² = 0.51) and was therefore selected for fine-tuning.


XGBoost Fine-Tuning

In [15]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

param_grid = {
    "learning_rate": [0.03, 0.05, 0.1],
    "max_depth": [4, 6, 8],
    "n_estimators": [300, 500, 800],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1, 3],
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring="neg_mean_squared_error"
)

grid_search.fit(X_train_sel, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score (MSE):", -grid_search.best_score_)

# Evaluate best model on DEV and TEST
best_model = grid_search.best_estimator_

pred_dev  = best_model.predict(X_dev_sel)
pred_test = best_model.predict(X_test_sel)

def _eval(name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    print(f"{name}: RMSE={rmse:.3f} | MAE={mae:.3f} | R2={r2:.3f}")

print("\nPerformance:")
_eval("DEV ", y_dev,  pred_dev)
_eval("TEST", y_test, pred_test)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
Best score (MSE): 283.16957600911456

Performance:
DEV : RMSE=16.693 | MAE=12.884 | R2=0.550
TEST: RMSE=17.414 | MAE=13.464 | R2=0.528



Although RandomForest initially achieved the highest R² on the training set (0.94), it showed signs of overfitting.
XGBoost demonstrated more balanced performance (Train R² = 0.74, Dev R² = 0.51) and was therefore selected for fine-tuning.
After hyperparameter optimization, XGBoost's performance improved substantially - R² increased from 0.51 to 0.55 - with consistent results across DEV and TEST sets, indicating good generalization and no overfitting.

In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os
save_dir = "/content/drive/MyDrive/Spotify_Project/6_Modeling"
os.makedirs(save_dir, exist_ok=True)

best_model.save_model(f"{save_dir}/xgb_finetuned_model3.json")
print("Fine-tuned XGBoost saved to:", f"{save_dir}/xgb_finetuned_model3.json")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fine-tuned XGBoost saved to: /content/drive/MyDrive/Spotify_Project/6_Modeling/xgb_finetuned_model3.json
